In [1]:
%pip install torch torchvision torchaudio tqdm scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import torch
import argparse
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms
from sklearn.metrics import accuracy_score
import numpy as np
from src.dataset import VSLR_Dataset


In [ ]:
class Args:
    data_path = "dataset/alphabet"
    image_size = 224
    num_epochs = 10
    batch_size = 32
    learning_rate = 0.001
    tensorboard_dir = "tensorboard"
    checkpoint_dir = "trained_models"
    checkpoint = None

args = Args()


In [4]:
os.makedirs(args.checkpoint_dir, exist_ok=True)
os.makedirs(args.tensorboard_dir, exist_ok=True)

writer = SummaryWriter(args.tensorboard_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((args.image_size, args.image_size), antialias=True),
    transforms.ToTensor()
])

In [5]:
train_set = VSLR_Dataset(args.data_path, transform=transform, mode="train")
val_set = VSLR_Dataset(args.data_path, transform=transform, mode="val")
test_set = VSLR_Dataset(args.data_path, transform=transform, mode="test")

print(f"Training set: {len(train_set)} images")
print(f"Validation set: {len(val_set)} images")
print(f"Test set: {len(test_set)} images")

train_dataloader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, num_workers=0)

Training set: 4160 images
Validation set: 520 images
Test set: 520 images


In [ ]:
num_class = train_set.num_class
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_class)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
criterion = nn.CrossEntropyLoss()

start_epoch = 0
best_acc = 0.0

if args.checkpoint and os.path.isfile(args.checkpoint):
    checkpoint = torch.load(args.checkpoint)
    start_epoch = checkpoint["epoch"]
    best_acc = checkpoint["best_acc"]
    model.load_state_dict(checkpoint["model_params"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    print(f"Loaded checkpoint: {args.checkpoint}")

for epoch in range(start_epoch, args.num_epochs):
    model.train()
    train_losses = []
    train_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{args.num_epochs}", colour="cyan")

    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_bar.set_postfix(loss=np.mean(train_losses))

    # Validation
    model.eval()
    val_losses = []
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_losses.append(loss.item())
            preds = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.tolist())
            all_preds.extend(preds.tolist())

    val_acc = accuracy_score(all_labels, all_preds)
    val_loss = np.mean(val_losses)

    print(f"Epoch {epoch+1}: Val Accuracy: {val_acc:.4f}, Val Loss: {val_loss:.4f}")
    writer.add_scalar("Val/Accuracy", val_acc, epoch)
    writer.add_scalar("Val/Loss", val_loss, epoch)

    # Save latest model
    checkpoint = {
        "epoch": epoch + 1,
        "best_acc": best_acc,
        "model_params": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, os.path.join(args.checkpoint_dir, "last.pt"))

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(checkpoint, os.path.join(args.checkpoint_dir, "best.pt"))
        print("=> New best model saved!")

writer.close()

Epoch 1/10: 100%|██████████| 130/130 [17:10<00:00,  7.93s/it, loss=0.268]


Epoch 1: Val Accuracy: 0.9885, Val Loss: 0.0624
=> New best model saved!


Epoch 2/10: 100%|██████████| 130/130 [18:40<00:00,  8.62s/it, loss=0.0847]


Epoch 2: Val Accuracy: 1.0000, Val Loss: 0.0016
=> New best model saved!


Epoch 3/10:  65%|██████▌   | 85/130 [14:00<07:26,  9.92s/it, loss=0.00865]